# Collect Traffic data

In [2]:

## Import libraries
import pandas as pd
import geopandas as gpd
import requests
import matplotlib.pyplot as plt

## Data Dictionary

Traffic data is collected from the following sources:
https://gis-txdot.opendata.arcgis.com/datasets/TXDOT::txdot-annual-average-daily-traffic-counts-public/api

The Data Dictionary for the Traffic data is as follows:

| Field Name | Description | Data Type | Example |
| ---------- | ----------- | --------- | ------- |
| DIST_NBR | District Number | Number | 1 |
| DIST_NM | District Name | Text | Amarillo |
| CNTY_NBR | County Number | Number | 1 |
| CNTY_NM | County Name | Text | Armstrong |
| TRFC_STATN_ID | Traffic Station ID | Number | 1 |
| CATEGORY | Category | Text | TOLL |
| ACTIVE | Active | Text | 1 |
| AADT_RPT_YR | Average Annual Daily Travel Report Year | Number | 2019 |
| AADT_RPT_QTY | Average Annual Daily Travel Report Quantity | Number | 1000 |
| AADT_RPT_HIST_01_QTY | Average Annual Daily Travel Report History 01 Quantity | Number | 1000 |
| AADT_RPT_HIST_'N'_QTY | Average Annual Daily Travel Report History 'N' Quantity , *where N is a number between 2 and 19*| Number | 1000 | 
| ON_ROAD | On Road | Text | FM 1912 |
| COUNT_CYCLE | Count Cycle | Text | 1 |
| LATITUDE | Latitude | Number | 35.2 |
| LONGITUDE | Longitude | Number | -101.8 |
| POINT | Point | Geometry | {"x": -101.8, "y": 35.2} |

The Data Dictionary for the Top 100 Roads data is as follows:
https://gis-txdot.opendata.arcgis.com/datasets/TXDOT::txdot-top-100-congested-roadways/about

| Field Name | Description | Data Type | Example |
| ---------- | ----------- | --------- | ------- |
| SEG_ID | Segment ID | Number | 3000001 |
| RANK | Rank | Number | 1 |
| TRK_RANK | Truck Rank | Number | 1 |
| RD_NM | Road Name | Text | IH 610 |
| DLAY_MILE | Delay Mile | Number | 384374 |
| TCI | TCI | Number | 1.5 |
| COST_DLAY | Total Est. Cost of Delay by Hour | Number | 43805069 |
| DIST_NM | District Name | Text | AUSTIN |
| YR | Year | Number | 2019 |
| TRK_DLAY | Truck Delay | Number | 136148 |
| COST_TRK | Cost Truck | Number | 1550000 |
| FID | FID | Number | 48 |
| Shape_Leng | Shape Length | Number | 0.061244 |
| Shape__Length | Shape Length | Number | 384374.0000000001 |
| geometry | Geometry | Geometry | POINTS |


Commuting data is collected from the following sources:
https://api.census.gov/data/2023/acs/acs1/subject?get=group(S0801)&ucgid=0500000US48021,0500000US48209,0500000US48453,0500000US48491

The Data Dictionary for the Commuting data is as follows:

| Field Name | Description | Data Type | Example |
| ---------- | ----------- | --------- | ------- |
| GEO_ID | Geo ID | Text | 0500000US48021 |
| NAME | Name | Text | Brazoria County, Texas |
| MOT_POV | Mode of Transportation | Text | Car, Truck, or Van |
| MOT_MOE | Mode of Transportation Margin of Error | Text | Car, Truck, or Van |
| DRIVE_ALONE | Drive Alone | Number | 0.8 |
| DRIVE_ALONE_MOE | Drive Alone Margin of Error | Number | 0.1 |
| CARPOOL | Carpool | Number | 0.1 |
| CARPOOL_MOE | Carpool Margin of Error | Number | 0.1 |
| POW_INSTATE| PLACE OF WORK IN STATE | Number | 0.9 |
| POW_INSTATE_MOE | PLACE OF WORK IN STATE Margin of Error | Number | 0.1 |
| POW_INCOUNTY | PLACE OF WORK IN COUNTY | Number | 0.8 |
| POW_INCOUNTY_MOE | PLACE OF WORK IN COUNTY Margin of Error | Number | 0.1 |
| POW_OUTCOUNTY | PLACE OF WORK OUT COUNTY | Number | 0.1 |
| POW_OUTCOUNTY_MOE | PLACE OF WORK OUT COUNTY Margin of Error | Number | 0.1 |
| TIME_OF_DEP_0000_0459 | TIME OF DEPARTURE 0000-0459 | Number | 0.1 |
| TIME_OF_DEP_0500_0529 | TIME OF DEPARTURE 0500-0529 | Number | 0.1 |
| TIME_OF_DEP_0530_0559 | TIME OF DEPARTURE 0530-0559 | Number | 0.1 |
| TIME_OF_DEP_0600_0629 | TIME OF DEPARTURE 0600-0629 | Number | 0.1 |
| TIME_OF_DEP_0630_0659 | TIME OF DEPARTURE 0630-0659 | Number | 0.1 |
| TIME_OF_DEP_0700_0729 | TIME OF DEPARTURE 0700-0729 | Number | 0.1 |
| TIME_OF_DEP_0730_0759 | TIME OF DEPARTURE 0730-0759 | Number | 0.1 |
| TIME_OF_DEP_0800_0829 | TIME OF DEPARTURE 0800-0829 | Number | 0.1 |
| TIME_OF_DEP_0830_0859 | TIME OF DEPARTURE 0830-0859 | Number | 0.1 |
| TIME_OF_DEP_0900_2359 | TIME OF DEPARTURE 0900-2359 | Number | 0.1 |
| MEAN_TRAVEL_TIME_TO_WORK_MIN | MEAN TRAVEL TIME TO WORK MIN | Number | 30 |
| MEAN_TRAVEL_TIME_TO_WORK_MIN_MOE | MEAN TRAVEL TIME TO WORK MIN Margin of Error | Number | 5 |






In [3]:
## Function for converting json to geodataframe
def json_to_gdf(data):
    traffic_df = pd.DataFrame([feature['attributes'] for feature in data['features']])
    traffic_df['geometry'] = [feature['geometry'] for feature in data['features']]
    gdf = gpd.GeoDataFrame(traffic_df, geometry=gpd.points_from_xy([geom['x'] for geom in traffic_df['geometry']], [geom['y'] for geom in traffic_df['geometry']]))
    return gdf

## Traffic Data for Austin, Texas

In [4]:
# Import data

## TXDOT Open data portal Json
traffic_url = "https://services.arcgis.com/KTcxiTD9dsQw4r7Z/arcgis/rest/services/TxDOT_AADT_Annuals_(Public_View)/FeatureServer/0/query?where=CNTY_NM%20%3D%20'BASTROP'%20OR%20CNTY_NM%20%3D%20'HAYS'%20OR%20CNTY_NM%20%3D%20'TRAVIS'%20OR%20CNTY_NM%20%3D%20'WILLIAMSON'&outFields=*&outSR=4326&f=json"
r = requests.get(traffic_url)
traffic_data = r.json()

## Convert to geodataframe
traffic_gdf = json_to_gdf(traffic_data)

## Peek at the data
print(traffic_gdf.head())




   DIST_NBR DIST_NM  CNTY_NBR CNTY_NM TRFC_STATN_ID    CATEGORY  ACTIVE  \
0        14  Austin       106    Hays      106A4501        TOLL       1   
1        14  Austin       106    Hays         106D1  ANNUAL ACR       1   
2        14  Austin       106    Hays        106D10  ANNUAL ACR       1   
3        14  Austin       106    Hays        106D11  ANNUAL ACR       1   
4        14  Austin       106    Hays    106D11NBSR  ANNUAL ACR       1   

   AADT_RPT_YEAR  AADT_RPT_QTY  AADT_RPT_HIST_01_QTY  ...  \
0           2023         18661               17497.0  ...   
1           2023         38211               33197.0  ...   
2           2023          1107                1191.0  ...   
3           2023          3409                3201.0  ...   
4           2023          5498                5621.0  ...   

   AADT_RPT_HIST_16_QTY  AADT_RPT_HIST_17_QTY  AADT_RPT_HIST_18_QTY  \
0                   NaN                   NaN                   NaN   
1                   NaN                 

## Top 100 congested roads in Austin, Texas

In [5]:
# Import data from json

top100_url = "https://services.arcgis.com/KTcxiTD9dsQw4r7Z/arcgis/rest/services/TxDOT_Top_100_Congested_Roadways/FeatureServer/0/query?where=DIST_NM%20%3D%20'AUSTIN'&outFields=*&geometry=&geometryType=esriGeometryEnvelope&inSR=4326&spatialRel=esriSpatialRelIntersects&outSR=4326&f=json"
top100_r = requests.get(top100_url)
top100_data = top100_r.json()

## Convert to pandas dataframe
top100_df = pd.DataFrame([feature['attributes'] for feature in top100_data['features']])
top100_df['geometry'] = [feature['geometry'] for feature in top100_data['features']]

# Check if 'x' and 'y' keys exist in the geometry data
top100_gdf = gpd.GeoDataFrame(top100_df, geometry=gpd.points_from_xy(
	[geom['x'] if 'x' in geom else None for geom in top100_df['geometry']],
	[geom['y'] if 'y' in geom else None for geom in top100_df['geometry']]
))

## Peek at the data
print(top100_gdf.head())
                  

    SEG_ID  RANK  TRK_RANK  RD_NM  DLAY_MILE   TCI  COST_DLAY DIST_NM    YR  \
0  3000002    27        20  IH 35     384374  1.74   43805069  Austin  2024   
1  3000003    23        18  IH 35     410691  1.59   44679819  Austin  2024   
2  3000004     3         1  IH 35     904542  1.99  197166266  Austin  2024   
3  3000005    81        54  IH 35     173298  1.33   31177150  Austin  2024   
4  3000007    70        50  IH 35     193915  1.36   29668500  Austin  2024   

   TRK_DLY  COST_TRK  FID  Shape_Leng  Shape__Length     geometry  
0   136148   9288532   48    0.061244    7750.680946  POINT EMPTY  
1   134644   9169646   49    0.060459    7452.252365  POINT EMPTY  
2   569826  39603864   50    0.117936   14820.832344  POINT EMPTY  
3   113217   7633408   51    0.095834   12052.722435  POINT EMPTY  
4   101568   6846945   52    0.081127   10378.446765  POINT EMPTY  


## Commuting Data for Austin, Texas

In [6]:
## API Data Link from Census Bureau
census_url = "https://api.census.gov/data/2023/acs/acs1/subject?get=group(S0801)&ucgid=0500000US48021,0500000US48209,0500000US48453,0500000US48491"
census_r = requests.get(census_url)
census_data = census_r.json()

## Convert to pandas dataframe
census_df = pd.DataFrame(census_data[1:], columns=census_data[0])

## Peek at the data
census_df.head()

                          

,GEO_ID,NAME,S0801_C01_001E,S0801_C01_001EA,S0801_C01_001M,S0801_C01_001MA,S0801_C01_002E,S0801_C01_002EA,S0801_C01_002M,S0801_C01_002MA,...,S0801_C03_055MA,S0801_C03_056E,S0801_C03_056EA,S0801_C03_056M,S0801_C03_056MA,S0801_C03_057E,S0801_C03_057EA,S0801_C03_057M,S0801_C03_057MA,ucgid
0,0500000US48021,"Bastrop County, Texas",48977,None,2193,None,77.1,None,4.6,None,...,(X),-888888888,(X),-888888888,(X),-888888888,(X),-888888888,(X),0500000US48021
1,0500000US48209,"Hays County, Texas",149429,None,4342,None,77.0,None,2.1,None,...,(X),-888888888,(X),-888888888,(X),-888888888,(X),-888888888,(X),0500000US48209
2,0500000US48453,"Travis County, Texas",766746,None,8528,None,67.3,None,1.1,None,...,(X),-888888888,(X),-888888888,(X),-888888888,(X),-888888888,(X),0500000US48453
3,0500000US48491,"Williamson County, Texas",376321,None,6397,None,72.7,None,1.5,None,...,(X),-888888888,(X),-888888888,(X),-888888888,(X),-888888888,(X),0500000US48491


In [7]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(census_df):
    # Drop columns with missing values
    census_df = census_df.dropna(axis=1)
    # Rename column 'S0801_C01_002E' to 'MOT_POV'
    census_df = census_df.rename(columns={'S0801_C01_002E': 'MOT_POV'})
    # Rename column 'S0801_C01_002M' to 'MOT MOE'
    census_df = census_df.rename(columns={'S0801_C01_002M': 'MOT_MOE'})
    # Rename column 'S0801_C01_003E' to 'DRIVE_ALONE'
    census_df = census_df.rename(columns={'S0801_C01_003E': 'DRIVE_ALONE'})
    # Rename column 'S0801_C01_003M' to 'DRIVE_ALONE_MOE'
    census_df = census_df.rename(columns={'S0801_C01_003M': 'DRIVE_ALONE_MOE'})
    # Rename column 'S0801_C01_008E' to 'WORKERS_PER_POV'
    census_df = census_df.rename(columns={'S0801_C01_008E': 'WORKERS_PER_POV'})
    # Drop columns: 'S0801_C01_009E', 'S0801_C01_009M' and 8 other columns
    census_df = census_df.drop(columns=['S0801_C01_009E', 'S0801_C01_009M', 'S0801_C01_010E', 'S0801_C01_010M', 'S0801_C01_011E', 'S0801_C01_013M', 'S0801_C01_013E', 'S0801_C01_012M', 'S0801_C01_011M', 'S0801_C01_012E'])
    # Rename column 'S0801_C01_014E' to 'POW_INSTATE'
    census_df = census_df.rename(columns={'S0801_C01_014E': 'POW_INSTATE'})
    # Drop columns: 'S0801_C01_026M', 'S0801_C01_017E' and 26 other columns
    census_df = census_df.drop(columns=['S0801_C01_026M', 'S0801_C01_017E', 'S0801_C01_017M', 'S0801_C01_018E', 'S0801_C01_018M', 'S0801_C01_019E', 'S0801_C01_019M', 'S0801_C01_020E', 'S0801_C01_020M', 'S0801_C01_021E', 'S0801_C01_021M', 'S0801_C01_022E', 'S0801_C01_022EA', 'S0801_C01_022M', 'S0801_C01_022MA', 'S0801_C01_023E', 'S0801_C01_023EA', 'S0801_C01_023M', 'S0801_C01_023MA', 'S0801_C01_024E', 'S0801_C01_024EA', 'S0801_C01_024M', 'S0801_C01_024MA', 'S0801_C01_025E', 'S0801_C01_025EA', 'S0801_C01_025M', 'S0801_C01_025MA', 'S0801_C01_026E'])
    # Rename column 'S0801_C01_027E' to 'TIME OF DEP_0000_0459'
    census_df = census_df.rename(columns={'S0801_C01_027E': 'TIME OF DEP_0000_0459'})
    # Drop columns: 'S0801_C01_037E', 'S0801_C01_045M' and 16 other columns
    census_df = census_df.drop(columns=['S0801_C01_037E', 'S0801_C01_045M', 'S0801_C01_045E', 'S0801_C01_044M', 'S0801_C01_044E', 'S0801_C01_043M', 'S0801_C01_042M', 'S0801_C01_043E', 'S0801_C01_042E', 'S0801_C01_041M', 'S0801_C01_041E', 'S0801_C01_040M', 'S0801_C01_039M', 'S0801_C01_040E', 'S0801_C01_039E', 'S0801_C01_038M', 'S0801_C01_038E', 'S0801_C01_037M'])
    # Drop columns to the right of 'S0801_C01_046M'
    census_df = census_df.loc[:, : 'S0801_C01_046M']
    # Drop columns: 'S0801_C01_001E', 'S0801_C01_001M'
    census_df = census_df.drop(columns=['S0801_C01_001E', 'S0801_C01_001M'])
    # Rename column 'S0801_C01_004E' to 'CARPOOL'
    census_df = census_df.rename(columns={'S0801_C01_004E': 'CARPOOL'})
    # Rename column 'S0801_C01_004M' to 'CARPOOL_MOE'
    census_df = census_df.rename(columns={'S0801_C01_004M': 'CARPOOL_MOE'})
    # Drop columns: 'S0801_C01_005E', 'S0801_C01_005M' and 4 other columns
    census_df = census_df.drop(columns=['S0801_C01_005E', 'S0801_C01_005M', 'S0801_C01_006E', 'S0801_C01_006M', 'S0801_C01_007E', 'S0801_C01_007M'])
    # Drop columns: 'WORKERS_PER_POV', 'S0801_C01_008M'
    census_df = census_df.drop(columns=['WORKERS_PER_POV', 'S0801_C01_008M'])
    # Rename column 'S0801_C01_014M' to 'POW_INSTATE_MOE'
    census_df = census_df.rename(columns={'S0801_C01_014M': 'POW_INSTATE_MOE'})
    # Rename column 'S0801_C01_015E' to 'POW_INCOUNTY'
    census_df = census_df.rename(columns={'S0801_C01_015E': 'POW_INCOUNTY'})
    # Rename column 'S0801_C01_015M' to 'POW_INCOUNTY_MOE'
    census_df = census_df.rename(columns={'S0801_C01_015M': 'POW_INCOUNTY_MOE'})
    # Rename column 'S0801_C01_016E' to 'POW_OUTCOUNTY'
    census_df = census_df.rename(columns={'S0801_C01_016E': 'POW_OUTCOUNTY'})
    # Rename column 'S0801_C01_016M' to 'POW_OUTCOUNTY_MOE'
    census_df = census_df.rename(columns={'S0801_C01_016M': 'POW_OUTCOUNTY_MOE'})
    # Drop columns: 'S0801_C01_027M', 'S0801_C01_028M' and 8 other columns
    census_df = census_df.drop(columns=['S0801_C01_027M', 'S0801_C01_028M', 'S0801_C01_029M', 'S0801_C01_030M', 'S0801_C01_031M', 'S0801_C01_032M', 'S0801_C01_033M', 'S0801_C01_034M', 'S0801_C01_035M', 'S0801_C01_036M'])
    # Rename column 'TIME OF DEP_0000_0459' to 'TIME_OF_DEP_0000_0459'
    census_df = census_df.rename(columns={'TIME OF DEP_0000_0459': 'TIME_OF_DEP_0000_0459'})
    # Rename column 'S0801_C01_028E' to 'TIME_OF_DEP_0500_0529'
    census_df = census_df.rename(columns={'S0801_C01_028E': 'TIME_OF_DEP_0500_0529'})
    # Rename column 'S0801_C01_029E' to 'TIME_OF_DEP_0530_0559'
    census_df = census_df.rename(columns={'S0801_C01_029E': 'TIME_OF_DEP_0530_0559'})
    # Rename column 'S0801_C01_030E' to 'TIME_OF_DEP_0600_0629'
    census_df = census_df.rename(columns={'S0801_C01_030E': 'TIME_OF_DEP_0600_0629'})
    # Rename column 'S0801_C01_031E' to 'TIME_OF_DEP_0630_0659'
    census_df = census_df.rename(columns={'S0801_C01_031E': 'TIME_OF_DEP_0630_0659'})
    # Rename column 'S0801_C01_032E' to 'TIME_OF_DEP_0700_0729'
    census_df = census_df.rename(columns={'S0801_C01_032E': 'TIME_OF_DEP_0700_0729'})
    # Rename column 'S0801_C01_033E' to 'TIME_OF_DEP_0730_0759'
    census_df = census_df.rename(columns={'S0801_C01_033E': 'TIME_OF_DEP_0730_0759'})
    # Rename column 'S0801_C01_034E' to 'TIME_OF_DEP_0800_0829'
    census_df = census_df.rename(columns={'S0801_C01_034E': 'TIME_OF_DEP_0800_0829'})
    # Rename column 'S0801_C01_035E' to 'TIME_OF_DEP_0830_0859'
    census_df = census_df.rename(columns={'S0801_C01_035E': 'TIME_OF_DEP_0830_0859'})
    # Rename column 'S0801_C01_036E' to 'TIME_OF_DEP_0900_2359'
    census_df = census_df.rename(columns={'S0801_C01_036E': 'TIME_OF_DEP_0900_2359'})
    # Rename column 'S0801_C01_046E' to 'MEAN_TRAVEL_TIME_TO_WORK_MIN'
    census_df = census_df.rename(columns={'S0801_C01_046E': 'MEAN_TRAVEL_TIME_TO_WORK_MIN'})
    # Rename column 'S0801_C01_046M' to 'MEAN_TRAVEL_TIME_TO_WORK_MOE'
    census_df = census_df.rename(columns={'S0801_C01_046M': 'MEAN_TRAVEL_TIME_TO_WORK_MOE'})
    return census_df

census_df_clean = clean_data(census_df.copy())
census_df_clean.head()

,GEO_ID,NAME,MOT_POV,MOT_MOE,DRIVE_ALONE,DRIVE_ALONE_MOE,CARPOOL,CARPOOL_MOE,POW_INSTATE,POW_INSTATE_MOE,...,TIME_OF_DEP_0530_0559,TIME_OF_DEP_0600_0629,TIME_OF_DEP_0630_0659,TIME_OF_DEP_0700_0729,TIME_OF_DEP_0730_0759,TIME_OF_DEP_0800_0829,TIME_OF_DEP_0830_0859,TIME_OF_DEP_0900_2359,MEAN_TRAVEL_TIME_TO_WORK_MIN,MEAN_TRAVEL_TIME_TO_WORK_MOE
0,0500000US48021,"Bastrop County, Texas",77.1,4.6,70.6,4.6,6.5,2.5,99.3,0.8,...,11.8,11.8,13.6,11.0,4.9,12.8,1.0,16.9,35.0,2.5
1,0500000US48209,"Hays County, Texas",77.0,2.1,66.5,2.8,10.5,2.2,99.4,0.4,...,7.6,9.8,11.9,15.8,11.1,8.0,3.2,24.5,31.3,1.5
2,0500000US48453,"Travis County, Texas",67.3,1.1,60.4,1.4,6.9,0.9,99.5,0.2,...,3.4,7.1,7.2,12.7,12.0,15.2,8.2,28.1,24.6,0.6
3,0500000US48491,"Williamson County, Texas",72.7,1.5,64.9,1.7,7.8,0.9,99.4,0.3,...,5.1,6.4,9.9,14.1,11.0,12.5,6.5,26.9,29.0,1.0


## Value of Daily Traffic Delay in Austin, Texas

In [ ]:
## Create a 

# Save to CSV

In [8]:
# Save the data

## Save the data to a csv file
traffic_gdf.to_csv('../data/outputs/traffic_data.csv', index=False)
top100_gdf.to_csv('../data/outputs/top100_data.csv', index=False)
census_df_clean.to_csv('../data/outputs/census_data.csv', index=False)
